### Face Identification Example

The notebook shows how to use face recognition with Aana SDK. Face recognition uses three separate deployments:
1. **Face Detection**, which returns bounding boxes and face landmarks (keypoints) for each detected face
2. **Face Feature Extraction**, which for a given image and face landmarks returns a face feature that can be used to compare face similarities.
3. **Face Database**, which uses 1 and 2 to extract reference faces and populate a reference face database that can be used to search for known identities across image/video collections.

In [1]:
# For onnx GPU support (face detection model), execute this code
!pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

Looking in indexes: https://pypi.org/simple, https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.5/201.5 MB 8.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "4"  # replace with your GPU ID
os.environ[
    "HF_TOKEN"
] = "hf_YOUR_HF_TOKEN"  # replace with your token, required for downloading the model weights and reference face database.
HF_TOKEN = os.getenv(
    "HF_TOKEN"
)  # Ideally we set the token in the terminal. However, it appears to be more difficult to do in a dev container, so doing it like this for now.

#### Load the face detection deployment

In [4]:
from aana.deployments.aana_deployment_handle import AanaDeploymentHandle
from aana.deployments.face_detection_deployment import (
    FaceDetectorConfig,
    FaceDetectorDeployment,
)

from aana.sdk import AanaSDK


aana_app = AanaSDK()
aana_app.connect(show_logs=False)

# Instantiate and register the face detection deployment
face_detector_deployment = FaceDetectorDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.1},
    user_config=FaceDetectorConfig(
        nms_thresh=0.4,
        batch_size=4,
        input_size=640,
    ).model_dump(mode="json"),
)

aana_app.register_deployment("face_detector", face_detector_deployment, deploy=True)
facedetector_handle = await AanaDeploymentHandle.create("face_detector")

2024-07-19 07:34:22,373	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(
2024-07-19 07:34:24,464	WARNING api.py:432 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-07-19 07:34:24,592	WARNING api.py:432 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-07-19 07:34:24,636	WARNING api.py:432 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-07-19 07:34:24,660	INFO worker.py:1603 -- Connecting to existing Ray cluster at address: 172.17.0.3:64992

#### Load the face feature extraction deployment

In [5]:
from aana.deployments.face_featureextraction_deployment import (
    FacefeatureExtractorConfig,
    FacefeatureExtractorDeployment,
)

FACEFEATURE_MODEL = "ir_101_webface4M"  # Name of the face feature model to be used. This has to be the same one for face feature extraction deployment and reference face database.

# Instantiate and register the face feature extraction deployment
facefeat_extractor_deployment = FacefeatureExtractorDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.2},
    user_config=FacefeatureExtractorConfig(
        feature_extractor_name=FACEFEATURE_MODEL,
        min_face_norm=19.0,
    ).model_dump(mode="json"),
)

aana_app.register_deployment(
    "facefeat_extractor", facefeat_extractor_deployment, deploy=True
)
facefeat_handle = await AanaDeploymentHandle.create("facefeat_extractor")

2024-07-19 07:39:55,995	WARNING api.py:432 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
The new client HTTP config differs from the existing one in the following fields: ['location']. The new HTTP config is ignored.
2024-07-19 07:39:56,006	INFO handle.py:126 -- Created DeploymentHandle '6vzx4hup' for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor').
2024-07-19 07:39:56,008	INFO handle.py:126 -- Created DeploymentHandle 'kvt5cj7g' for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor').
2024-07-19 07:40:03,086	INFO handle.py:126 -- Created DeploymentHandle '41dftg96' for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor').
2024-07-19 07:40:03,088	INFO api.py:609 -- Deployed app 'facefeat_extractor' successfully.
2024-07-19 07:40:03,098	INFO handle.py:126 -- Created DeploymentHandle 'fzbq6xvw' for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor'

2024-07-19 07:40:03,111	INFO pow_2_scheduler.py:260 -- Got updated replicas for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor'): {'5hp92jto'}.
2024-07-19 07:40:14,074	INFO pow_2_scheduler.py:260 -- Got updated replicas for Deployment(name='FaceDatabaseDeployment', app='face_database'): {'363ljsgm'}.
2024-07-19 08:08:47,861	INFO pow_2_scheduler.py:260 -- Got updated replicas for Deployment(name='FaceDetectorDeployment', app='face_detector'): set().
2024-07-19 08:08:47,867	INFO pow_2_scheduler.py:260 -- Got updated replicas for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor'): set().
2024-07-19 08:08:47,871	INFO pow_2_scheduler.py:260 -- Got updated replicas for Deployment(name='FaceDatabaseDeployment', app='face_database'): set().


#### Load the reference face database deployment

In [6]:
import importlib

import aana.deployments.face_database_deployment

importlib.reload(aana.deployments.face_database_deployment)

from aana.configs.settings import settings
from aana.deployments.face_database_deployment import (
    FaceDatabaseConfig,
    FaceDatabaseDeployment,
)

face_database_deployment = FaceDatabaseDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.1},
    user_config=FaceDatabaseConfig(
        face_threshold=1.18,
        facenorm_threshold=18.0,
        face_features_directory=settings.artifacts_dir / "face_features_database",
        feature_extractor_name=FACEFEATURE_MODEL,
        hugging_face_token=HF_TOKEN,
    ).model_dump(mode="json"),
)

aana_app.register_deployment("face_database", face_database_deployment, deploy=True)

facedatabase_handle = await AanaDeploymentHandle.create("face_database")

2024-07-19 07:40:05,930	WARNING api.py:432 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-07-19 07:40:05,941	WARNING api.py:432 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
The new client HTTP config differs from the existing one in the following fields: ['location']. The new HTTP config is ignored.
2024-07-19 07:40:05,966	INFO handle.py:126 -- Created DeploymentHandle '7sko4d2y' for Deployment(name='FaceDatabaseDeployment', app='face_database').
2024-07-19 07:40:05,967	INFO handle.py:126 -- Created DeploymentHandle '0pggxfjm' for Deployment(name='FaceDatabaseDeployment', app='face_database').


2024-07-19 07:40:14,034	INFO handle.py:126 -- Created DeploymentHandle 'g95ewhoo' for Deployment(name='FaceDatabaseDeployment', app='face_database').
2024-07-19 07:40:14,036	INFO api.py:609 -- Deployed app 'face_database' successfully.
2024-07-19 07:40:14,062	INFO handle.py:126 -- Created DeploymentHandle 'evsfiw6w' for Deployment(name='FaceDatabaseDeployment', app='face_database').


### Test Deployments

#### Run face detection

In [8]:
from aana.core.models.image import Image
from pathlib import Path

image = Image(
    path=Path(
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000101.jpg"
    )
)

detector_output = await facedetector_handle.predict([image])

print(detector_output)

2024-07-19 07:40:37,659	INFO handle.py:126 -- Created DeploymentHandle '27g5cj0f' for Deployment(name='FaceDetectorDeployment', app='face_detector').


{'bounding_boxes': [array([[415.59457   ,  96.23158   , 481.6673    , 184.54308   ,
          0.87722456],
       [188.31061   ,  60.958458  , 272.11404   , 174.89207   ,
          0.7909324 ]], dtype=float32)], 'keypoints': [array([[[443.3935 , 131.3212 ],
        [471.51398, 133.13217],
        [462.67175, 147.79501],
        [447.37967, 164.56483],
        [467.67795, 165.76624]],

       [[232.06563, 103.42656],
        [264.94907, 106.04597],
        [261.4791 , 123.83058],
        [233.09895, 144.04256],
        [261.0374 , 146.01912]]], dtype=float32)]}


In [9]:
images = [
    Image(
        path=Path(
            "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000101.jpg"
        )
    ),
    Image(
        path=Path(
            "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000102.jpg"
        )
    ),
    Image(
        path=Path(
            "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000103.jpg"
        )
    ),
    Image(
        path=Path(
            "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000104.jpg"
        )
    ),
    Image(
        path=Path(
            "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000105.jpg"
        )
    ),
    Image(
        path=Path(
            "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000102.jpg"
        )
    ),
    Image(
        path=Path(
            "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000103.jpg"
        )
    ),
    Image(
        path=Path(
            "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000104.jpg"
        )
    ),
    Image(
        path=Path(
            "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000105.jpg"
        )
    ),
    Image(
        path=Path(
            "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000106.jpg"
        )
    ),
]

detector_output = await facedetector_handle.predict(images)

print(detector_output)

2024-07-19 07:40:56,988	INFO handle.py:126 -- Created DeploymentHandle '4wz6op0p' for Deployment(name='FaceDetectorDeployment', app='face_detector').


{'bounding_boxes': [array([[415.59457   ,  96.23158   , 481.6673    , 184.54308   ,
          0.87722456],
       [188.31061   ,  60.958458  , 272.11404   , 174.89207   ,
          0.7909324 ]], dtype=float32), array([[418.13217   ,  91.993866  , 484.82758   , 182.29898   ,
          0.86423916],
       [174.74506   ,  57.504745  , 258.35153   , 175.60551   ,
          0.79931784]], dtype=float32), array([[409.99353   ,  92.0741    , 477.31976   , 182.75638   ,
          0.86028343],
       [170.32983   ,  63.222565  , 253.86038   , 180.34108   ,
          0.77052784]], dtype=float32), array([[402.97488   ,  91.797035  , 470.39627   , 182.33727   ,
          0.8626434 ],
       [176.70801   ,  65.474434  , 262.6093    , 185.3455    ,
          0.80597097]], dtype=float32), array([[409.70914   ,  94.12892   , 477.73315   , 184.83496   ,
          0.86796105],
       [194.3403    ,  62.5718    , 283.30243   , 181.06514   ,
          0.7984799 ]], dtype=float32), array([[418.13217   ,  91

#### Run Face feature extraction using the output of the face detector

In [10]:
keypoints = detector_output["keypoints"]
facefeat_output = await facefeat_handle.predict(images, keypoints)

print(facefeat_output)

2024-07-19 07:41:02,683	INFO handle.py:126 -- Created DeploymentHandle 'e438l24a' for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor').


{'facefeats_per_image': [{'face_feats': array([[-0.04401124, -0.0272526 ,  0.01495183, ..., -0.06465618,
         0.0389987 ,  0.0128512 ],
       [ 0.01096947, -0.02446904,  0.03266014, ..., -0.01851118,
         0.01181385,  0.02578571]], dtype=float32), 'norms': array([[21.607306],
       [23.78863 ]], dtype=float32)}, {'face_feats': array([[-0.03572392, -0.04950345,  0.00264802, ..., -0.05969051,
         0.02760378,  0.0097962 ],
       [ 0.02021521, -0.01210909,  0.04694788, ...,  0.00210422,
         0.00151184,  0.05000335]], dtype=float32), 'norms': array([[22.046799],
       [23.471266]], dtype=float32)}, {'face_feats': array([[-0.04562347, -0.0468936 , -0.00619584, ..., -0.06936102,
         0.03672777, -0.00146778],
       [ 0.01352969, -0.01935948,  0.04992234, ..., -0.00231571,
         0.00225516,  0.03048169]], dtype=float32), 'norms': array([[21.70192 ],
       [23.681787]], dtype=float32)}, {'face_feats': array([[-0.05637712, -0.04676075,  0.0011431 , ..., -0.07301167

#### Search the reference face database with the face features we extracted above

In [11]:
all_identities_in_database = await facedatabase_handle.get_all_identities()

print(
    "There are {} identities in the reference face database.".format(
        len(all_identities_in_database)
    )
)
print("The first 10 are: {}".format(all_identities_in_database[0:10]))

2024-07-19 07:41:30,472	INFO handle.py:126 -- Created DeploymentHandle '2l0p87i6' for Deployment(name='FaceDatabaseDeployment', app='face_database').


There are 32898 identities in the reference face database.
The first 10 are: ['Paola Minaccioni', 'Yann Hnautra', 'Anna Raadsveld', 'Charlotte Arnold', 'Dustin Clare', 'Erica Carroll', 'J.A. Bayona', 'Kathryn Bigelow', 'Yon González', 'Gil Darnell']


In [12]:
recognized_faces = await facedatabase_handle.identify_faces(
    facefeat_output["facefeats_per_image"]
)
print(recognized_faces)

2024-07-19 07:41:33,184	INFO handle.py:126 -- Created DeploymentHandle 'ulr0xxl0' for Deployment(name='FaceDatabaseDeployment', app='face_database').


{'identities_per_image': [[{'person_id': 'Donna Murphy', 'image_id': '587eed19-0fd7-4f1b-8911-e347f86b1bc8', 'distance': 0.7868431806564331, 'norm': 21.607306, 'quality': 'good'}, {'person_id': 'Patrick Stewart', 'image_id': '00a35c5b-aacf-4574-8219-9c4cb36486a4', 'distance': 0.7150369882583618, 'norm': 23.78863, 'quality': 'good'}], [{'person_id': 'Donna Murphy', 'image_id': '587eed19-0fd7-4f1b-8911-e347f86b1bc8', 'distance': 0.8411625623703003, 'norm': 22.046799, 'quality': 'good'}, {'person_id': 'Patrick Stewart', 'image_id': '00a35c5b-aacf-4574-8219-9c4cb36486a4', 'distance': 0.6993443965911865, 'norm': 23.471266, 'quality': 'good'}], [{'person_id': 'Donna Murphy', 'image_id': '587eed19-0fd7-4f1b-8911-e347f86b1bc8', 'distance': 0.8204156756401062, 'norm': 21.70192, 'quality': 'good'}, {'person_id': 'Patrick Stewart', 'image_id': '00a35c5b-aacf-4574-8219-9c4cb36486a4', 'distance': 0.6765266060829163, 'norm': 23.681787, 'quality': 'good'}], [{'person_id': 'Donna Murphy', 'image_id': 

## DEBUG (Ignore anything below here)

In [10]:
from aana.configs.settings import settings
import importlib
import aana.deployments.face_database_deployment

importlib.reload(aana.deployments.face_database_deployment)
from aana.deployments.face_database_deployment import (
    FaceDatabaseConfig,
    FaceDatabaseDeployment,
)


facedatabase_config = FaceDatabaseConfig(
    face_threshold=1.18,
    facenorm_threshold=19.0,
    face_features_directory=settings.artifacts_dir / "face_features_database",
    feature_extractor_name=FACEFEATURE_MODEL,
)
facedatabase_deployment = FaceDatabaseDeployment()
await facedatabase_deployment.apply_config(facedatabase_config.model_dump(mode="json"))

### SDK endpoints
Run the SDK by running the following command in the terminal:
`CUDA_VISIBLE_DEVICES=4 python aana/projects/face_recognition/app.py`

In [42]:
import requests, json


def identify_faces(im_paths):
    """Identify faces in images."""

    data = {"images": [{"path": im_path} for im_path in im_paths]}

    response = requests.post(
        "http://127.0.0.1:8000/recognize_faces",
        data={"body": json.dumps(data)},
        stream=False,
    )
    res = response.json()

    return res

In [44]:
res = identify_faces(
    [
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000000.jpg",
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000001.jpg",
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000002.jpg",
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000003.jpg",
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000004.jpg",
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000005.jpg",
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000006.jpg",
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000007.jpg",
    ]
)
print(res)

{'identified_faces_per_image': ['No faces identified', 'No faces identified', 'No faces identified', 'No faces identified', 'No faces identified', [{'person_name': 'Patrick Stewart', 'image_id': '00a35c5b-aacf-4574-8219-9c4cb36486a4', 'distance': 1.1752872467041016, 'norm': 20.057465, 'quality': 'good', 'bbox_xyxy': [181.73259, 142.26784, 221.9567, 204.69104]}], 'No faces identified', 'No faces identified']}


In [35]:
from collections import defaultdict


def parse_faceresults_images(faceid_output, bboxes_per_frame, accept_bad_faces=False):
    """Parse the output of the faceid model.

    Parameters
    ----------
    faceid_output : dict
        Output of the faceid model.
    accept_bad_faces : bool, optional
        If True, accept faces with low quality. The default is False.

    Returns:
    -------
    identities_timestamped : dict
        Dictionary with identities as keys and a dict with timestamps, bbox_xyxy, distance, face_norm and quality as values.
    """
    identities_per_frame = []
    for frame_id, frame_result in enumerate(faceid_output):
        if frame_result == "No faces identified":
            identities_per_frame.append("No faces identified")
        else:
            ids_curr_frame = []
            for face_number, face_ in enumerate(frame_result):
                if face_["person_name"] != "unknown" and (
                    face_["quality"] == "good" or accept_bad_faces
                ):
                    ids_curr_frame.append(
                        {
                            "person_name": face_["person_name"],
                            "image_id": face_["image_id"],
                            "distance": face_["distance"],
                            "norm": face_["norm"],
                            "quality": face_["quality"],
                            "bbox_xyxy": bboxes_per_frame[frame_id][face_number][0:4],
                        }
                    )
            if len(ids_curr_frame) > 0:
                identities_per_frame.append(ids_curr_frame)
            else:
                identities_per_frame.append("No faces identified")

    return identities_per_frame


out = parse_faceresults_images(res["identified_faces"], res["bboxes"])
print(out)

['No faces identified', 'No faces identified', 'No faces identified', 'No faces identified', 'No faces identified', [{'person_name': 'Patrick Stewart', 'image_id': '00a35c5b-aacf-4574-8219-9c4cb36486a4', 'distance': 1.1752872467041016, 'norm': 20.057465, 'quality': 'good', 'bbox_xyxy': [181.73259, 142.26784, 221.9567, 204.69104]}], 'No faces identified', 'No faces identified']


In [27]:
res["identified_faces"][5][0]["toto"] = 5

#### Run Face Recognition on Video

In [12]:
import json
import requests

youtube_id = "wxN1T1uxQ2g"

video = {
    "url": "https://www.youtube.com/watch?v={}".format(
        youtube_id
    ),  # Video URL, Aana SDK supports URLs (including YouTube), file paths or even raw video data
    "media_id": youtube_id,  # Media ID, so we can ask questions about the video later by using this ID
}

data = {
    "video_params": {
        "fast_mode_enabled": True,  # Enable fast mode, which only processes keyframes
    },
    "video": video,
}

url = "http://127.0.0.1:8000/recognize_faces_video"
response = requests.post(url, data={"body": json.dumps(data)})

res = response.json()

In [11]:
res

{'recognized_persons': {'Michelle Yeoh': [{'timestamp': 3.7950000762939453,
    'bbox_xyxy': [756.49927, 225.14713, 1085.705, 655.3115],
    'distance': 1.0151426792144775,
    'face_norm': 20.901358,
    'quality': 'good'},
   {'timestamp': 4.086999893188477,
    'bbox_xyxy': [773.77936, 216.38254, 1110.3029, 661.1126],
    'distance': 1.0362409353256226,
    'face_norm': 20.806562,
    'quality': 'good'},
   {'timestamp': 4.379000186920166,
    'bbox_xyxy': [821.3978, 216.20444, 1162.6763, 669.901],
    'distance': 1.0562446117401123,
    'face_norm': 20.891169,
    'quality': 'good'},
   {'timestamp': 4.671000003814697,
    'bbox_xyxy': [863.3463, 209.61104, 1210.526, 669.2429],
    'distance': 1.060685396194458,
    'face_norm': 20.877834,
    'quality': 'good'},
   {'timestamp': 4.9629998207092285,
    'bbox_xyxy': [896.6911, 211.92531, 1248.185, 668.13293],
    'distance': 1.0712049007415771,
    'face_norm': 21.148489,
    'quality': 'good'},
   {'timestamp': 5.255000114440918,
